In [7]:
!pip install openai==1.13.3


In [8]:
import asyncio
from openai import AsyncAzureOpenAI

In [9]:
import requests

In [10]:
file_name = 'system.txt'

with open(file_name, 'w') as file:
    pass

print(f"Created file: {file_name}")

Created file: system.txt


In [11]:
# Configuration settings
azure_oai_endpoint ="https://eyuser27.openai.azure.com/"
azure_oai_key ="98820bac08d241298094de8a35be4b10"
azure_oai_deployment ="gpt-35-turbo-16k"

# Configure the Azure OpenAI client
client = AsyncAzureOpenAI(
    azure_endpoint=azure_oai_endpoint,
    api_key=azure_oai_key,
    api_version="2024-02-15-preview"
)

In [12]:
async def main():
    try:
        # Write system message to file and read the content to system_text variable
        with open(file="/content/system.txt", mode="w", encoding="utf8") as file:
            file.write(
                "You are a Medical practitioner assistant. You help with giving appointments on the requesting day for regular patients and for others if they dont specify the date give them after 45 days , "
                "after getting full name and date, please proceed with appointment conirmation message"
                "giving general information about the medical field. You write in a professional tone "
                "but can tailor your writing style to best work for a medical professional. If you do not know the "
                "answer to a question, respond by saying - I do not know how to respond to your query, kindly visit "
                "our clinic for more information."
            )
        system_text = open(file="/content/system.txt", encoding="utf8").read()

        inititialInput = "Hello there,\nThis is Helixa from Helio Health Care, \nI can give information about Helio Health Care\nType 'quit' to exit the program.\n\nHow may I help you? : "
        user_text = input(inititialInput)

        while True:
            # Once the conversation has started, no need to show the initial input
            if user_text.lower() == 'quit':
                print('Exiting the bot, thank you...')
                break
            # Check if the user is requesting an appointment
            if "appointment" in user_text.lower() or "book" in user_text.lower():
                # Confirm the appointment
                confirmation_message = "Your appointment has been booked successfully."
                print(f"Helixa: {confirmation_message}")

                # Generate an "appointment booked" image
                image_prompt = "A confirmation image for a booked appointment with Helio Health Care hospital. The image should include a calendar, a checkmark, and the text 'Appointment Confirmed' with the Helio Health Care logo."
                image_url = await generate_image(image_prompt)
                print(f"Helixa: Here is your appointment confirmation image: {image_url}")
                break;
            else:
                response = await call_openai_model(system_message=system_text,
                                                   user_message=user_text,
                                                   model=azure_oai_deployment,
                                                   client=client)


            user_text = input("\nHow else can I assist you? : ")
            await call_openai_model(system_message=system_text,
                                    user_message=user_text,
                                    model=azure_oai_deployment,
                                    client=client)

            user_text = input("\nHow else can I assist you? : ")

    except Exception as ex:
        print(ex)

In [13]:
grounding_text = """
---
Helio Health Care Inc is a premier provider of comprehensive health care services dedicated to improving the health and well-being of our community. We strive to deliver high-quality, patient-centered care through our extensive network of medical professionals, state-of-the-art facilities, and innovative health care solutions.

Our Mission
At Helio Health Care Inc, our mission is to enhance the quality of life for our patients by providing accessible, compassionate, and personalized health care services. We are committed to promoting wellness, preventing illness, and managing chronic conditions effectively.

Our Services
We offer a wide range of health care services to meet the diverse needs of our patients, including:

Primary Care: Our primary care physicians are dedicated to building long-term relationships with patients, focusing on preventive care, routine check-ups, and the management of chronic conditions.
Specialty Care: We provide specialized medical services in areas such as cardiology, oncology, orthopedics, neurology, and more, ensuring expert care for complex health issues.
Emergency and Urgent Care: Our emergency departments are equipped to handle medical emergencies with speed and efficiency, while our urgent care centers provide immediate attention for non-life-threatening conditions.
Pediatrics: We offer comprehensive pediatric care, from newborns to adolescents, ensuring the health and development of our youngest patients.
Women's Health: Our dedicated team provides a full spectrum of women's health services, including gynecology, obstetrics, and reproductive health.
Mental Health: We recognize the importance of mental health and provide counseling, psychiatric services, and support for various mental health conditions.

Patient Experience
At Helio Health Care Inc, patient satisfaction is our top priority. We are dedicated to providing a seamless and positive experience at every touchpoint, from appointment scheduling and medical consultations to follow-up care and support services.
---

"""

with open('grounding.txt', 'w') as file:
    file.write(grounding_text)

print("Created file: grounding.txt")


Created file: grounding.txt


In [14]:
async def generate_image(prompt):
    azure_endpoint = "https://eyuser3.openai.azure.com/"
    api_key="06f6ae63b2f94f23b2c39ec67a612ee8"
    api_version="2024-02-15-preview"
    try:
        url = "{}openai/deployments/dalle3/images/generations?api-version={}".format(azure_endpoint, api_version)
        headers= { "api-key": api_key, "Content-Type": "application/json" }
        body = {
            "prompt": prompt,
            "n": 1,
            "size": "1024x1024"
             }
        response = requests.post(url, headers=headers, json=body)
        print(response.text)
        image_url = response.json()['data'][0]['url']

        return image_url
    except Exception as e:
        return f"An error occurred: {e}"

In [15]:
async def call_openai_model(system_message, user_message, model, client):
    # Format and send the request to the model
    print("Adding grounding context from grounding.txt")
    grounding_text = open(file="/content/grounding.txt", encoding="utf8").read().strip()
    user_message = grounding_text + user_message
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    print("Sending request to Azure OpenAI model...")
    # Call the Azure OpenAI model
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.9,
        max_tokens=1000
    )



    print("Response:" + response.choices[0].message.content + "\n")

In [16]:
await main()

Hello there,
This is Helixa from Helio Health Care, 
I can give information about Helio Health Care
Type 'quit' to exit the program.

How may I help you? : I need an appointment on August 5
Helixa: Your appointment has been booked successfully.
{"created":1718197968,"data":[{"content_filter_results":{"hate":{"filtered":false,"severity":"safe"},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"prompt_filter_results":{"hate":{"filtered":false,"severity":"safe"},"profanity":{"detected":false,"filtered":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":false,"severity":"safe"}},"revised_prompt":"An image representing a confirmed appointment for a generic healthcare center. The image features a calendar indicating a specific date, a green checkmark signifying confirmation, and the text 'Appointment Confirmed' prominently di